# Katakit Health Risk Assessment Predictor
### —based on mDoc Data 
ِ_Authors: Youssef Abdelkader, Aamen Muharram, Ela Ben Saad, Aya Ben Saad, Amine Oueslati_ 
<br>

## Abstract

Using relevant factors we will be calculating health risk factors to identify patients with a high risk of Cardiovascular within the next 10 years. The algorithm will be used mainly to raise awareness and alarm the patient to be more cautious about their health and potentially follow the recommendations doctors will give. <br><br>
Addressing this problem we worked with mDoc dataset and extracted the essential features which are:
+ Age
+ Gender
+ BMI
+ Systolic Blood Pressure
+ Blood Pressure Medications (Hypertension treatment)
+ Smoking Status
+ Diabetes Status
 
Based on these features, our algorithm will calculate the Health Risk Assessment.<br><br>
This algorithm is based on the Framingham Health Risk Assessment Algorithm. Throughout this notebook we will be explaining the process of cleaning the data, reaching our final features, how did we choose theright algorithm to implement, and finally how did we implement the algorithm on mDoc limited data with the relevant feature to calculate the HRA percentage.

In [13]:
import numpy as np
import pandas as pd 

df = pd.read_excel('ALA Hackathon Data.xlsx')
df.head()

,Id,age,Hypertension,Arthritis,Diabetes,Diabetes Type,Overweight/Obese,Headache/Migraine,Chronic Kidney Disease,Stroke,...,Drink Alcohol,Systolic Blood Pressure,Gender,Weight,Height,BMI,Triglycerides,HDL,LDL,Total Cholesterol
0,1,54,No,No,No,unknown,Yes,No,No,No,...,No,124.333333,female,65.0,1.608,25.14,NaN,NaN,NaN,NaN
1,2,62,Yes,No,Yes,unknown,Yes,No,No,No,...,No,124.666667,female,69.0,1.580,27.64,1.02,1.45,1.89,3.80
2,3,68,No,No,Yes,unknown,No,No,No,No,...,Yes,117.333333,male,49.0,1.560,20.13,NaN,NaN,NaN,NaN
3,4,47,No,No,Yes,unknown,Yes,No,No,No,...,No,132.000000,female,82.0,1.660,29.76,NaN,NaN,NaN,NaN
4,5,59,Yes,No,No,unknown,Yes,No,No,No,...,Yes,129.000000,male,90.0,1.730,30.07,2.46,1.07,4.40,6.59


#### Exploring the data from mDoc

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15036 entries, 0 to 15035
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Id                                15036 non-null  int64  
 1   age                               15036 non-null  int64  
 2   Hypertension                      15036 non-null  object 
 3   Arthritis                         15036 non-null  object 
 4   Diabetes                          15036 non-null  object 
 5   Diabetes Type                     15036 non-null  object 
 6   Overweight/Obese                  15036 non-null  object 
 7   Headache/Migraine                 15036 non-null  object 
 8   Chronic Kidney Disease            15036 non-null  object 
 9   Stroke                            15036 non-null  object 
 10  Heart Disease/Attack              15036 non-null  object 
 11  Taking Blood Pressure Medication  15036 non-null  object 
 12  Smok

The dataset contains many features that may be relevant

In [23]:
df.describe()

,Id,age,Systolic Blood Pressure,Weight,Height,BMI,Triglycerides,HDL,LDL,Total Cholesterol
count,15036.000000,15036.000000,15036.000000,15036.000000,15036.000000,15036.000000,20.00000,20.00000,20.000000,20.000000
mean,7518.500000,35.671788,117.092271,71.516617,1.622472,27.202572,12.29700,11.15250,25.542600,52.849500
std,4340.663659,8.736452,15.535114,13.704465,0.068915,5.168032,26.18351,24.89413,56.599062,98.217396
min,1.000000,25.000000,71.000000,34.900000,1.390000,12.010000,0.50000,0.61000,0.960000,2.900000
25%,3759.750000,30.000000,107.000000,62.000000,1.580000,23.500000,0.95500,1.05750,2.460000,4.140000
50%,7518.500000,34.000000,115.000000,70.000000,1.610000,26.300000,1.13000,1.46500,3.335000,5.240000
75%,11277.250000,39.000000,125.000000,79.000000,1.660000,30.100000,2.43750,1.83250,4.515000,8.242500
max,15036.000000,86.000000,246.500000,155.000000,2.240000,65.360000,88.80000,100.00000,181.430000,270.000000


### We seem to have a lack in the Cholesterol rates for every patient

From 15036 patient, only 20 of them have reported their Cholesterol rates.

Let's have a look on the reported data

In [10]:
df22[~df22.HDL.isna()]

,Id,age,Hypertension,Arthritis,Diabetes,Diabetes Type,Overweight/Obese,Headache/Migraine,Chronic Kidney Disease,Stroke,...,Drink Alcohol,Systolic Blood Pressure,Gender,Weight,Height,BMI,Triglycerides,HDL,LDL,Total Cholesterol
1,2,62,Yes,No,Yes,unknown,Yes,No,No,No,...,No,124.666667,female,69.00,1.58,27.64,1.02,1.45,1.890,3.80
4,5,59,Yes,No,No,unknown,Yes,No,No,No,...,Yes,129.000000,male,90.00,1.73,30.07,2.46,1.07,4.400,6.59
8,9,48,Yes,No,No,unknown,Yes,No,No,No,...,No,142.000000,female,75.00,1.62,28.58,1.00,1.49,2.470,4.41
10,11,74,Yes,No,Yes,unknown,No,No,No,No,...,No,123.666667,male,66.00,1.63,24.84,1.73,1.15,0.960,2.90
11,12,74,Yes,No,Yes,unknown,Yes,No,No,No,...,No,127.666667,female,78.00,1.50,34.67,2.43,0.61,2.430,4.14
12,13,65,Yes,No,Yes,unknown,Yes,No,No,No,...,No,122.333333,male,81.00,1.68,28.70,1.41,1.48,4.860,6.99
13,14,51,Yes,No,No,unknown,Yes,No,No,No,...,No,130.333333,female,71.00,1.63,26.72,0.99,0.99,2.760,4.20
18,19,65,No,No,Yes,unknown,No,No,No,No,...,No,149.333333,female,50.00,1.44,24.11,1.10,1.02,2.620,4.14
20,21,64,Yes,No,Yes,unknown,Yes,No,No,No,...,No,130.000000,female,93.00,1.60,36.33,0.88,1.41,3.370,5.18
25,26,84,Yes,No,Yes,unknown,Yes,No,No,No,...,No,121.000000,female,63.00,1.58,25.24,0.98,1.84,3.580,5.87


#### Getting the minimun and maximum age available in our dataset

In [19]:
min(df.age), max(df.age)

(25, 86)

Age is an inmportant factor in HRAs. Some algorithms do not apply in certain ages; therfore, we need to know our age range to best decide which algorithm will be best in our prediction.

#### Gender based assessment

Each algorithm reacts differently to different genders, therefore, we need to see which gender soverreigns the dataset

In [27]:
df.groupby('Gender')['Id'].count()

Gender
female    14045
male        991
Name: Id, dtype: int64

Female is the answer.

No Diabetes type declared

In [121]:
len(df['Diabetes Type'][df['Diabetes Type']!='unknown'])

0

## Limitations

After carefully analyzing the data from mDoc, we concluded the following:
+ mDoc is based in Kenya, Africa so the patients are African-based.
+ The age range is between 25 and 86 years old.
+ Most of the patient did not report their cholesterol rates or theit triglycerides. (only 0.13% did)
+ Most of the patient's data are for females. 
+ Diabetes type in not declared in any of the records.

## Comparing all the available algorithms

### Reynolds Risk Score 
This risk calculator relies on specific deciding factors which are age, blood pressure, cholesterol levels and whether you smoke or not, a blood test called hsCRP (a measure of inflammation), and whether or not either of your parents had a heart attack before they reached age 60 (a measure of genetic risk). Using these factors this calculator predicts the risk of having a stroke, heart disease, etc. what makes this algorithm unique is its use of the genetics factor.

Drawbacks:
+ Only for women
+ Only used and tested in North America (not relevant to the African continent)


<b>Result:<b><br>
Reynolds Risk Score will not be used

### ACC/AHA Cardiovascular Risk Calculator:
using multiple testing methods the ACC/AHA cardiovascular risk calculator was developed based on numerous cohorts with a demographic focus with a final result of a risk calculator for non-Hispanic African-American and non-Hispanic white men and women from 40 to 79 years of age.

Drawbacks:
+ Not enough data to predict for less than 40 or more than 75 (only works in the range of 40 - 75
+ Need the HDL and Total Cholesterol to work (which we lack of in the mDoc dataset)


<b>Result:<b><br>
Acc/AHA Risk Cardiovascular Risk will not be used

### Framingham General CVD Risk Profile

The Framingham Risk Score predicts only future coronary heart disease (CHD) events, however, it does not predict future total cardiovascular events, meaning that it does not predict risk for stroke, transient ischemic attack (TIA), and heart failure. These also important patient outcomes were included in the 2008 Framingham General Cardiovascular Risk Score.[2] The predicted risk for an individual usually is higher with the 2008 Framingham General Cardiovascular Risk Score than with the 2002 Framingham Risk Score.[citation needed] <br><br>

The Framingham Risk Score could overestimate (or underestimate) risk in populations other than Europeans and Africans. It is not yet clear if this limitation is real, or appears to be real because of differences in methodology, etc. As a result, other countries may prefer to use another risk score. <br><br>

<b>The data from mDoc is for African-based patients, in addition that it uses only the BMI indicator with no neeed of the cholesterol rates; therefore, Framingham General CVD Risk Profile is our best choice. <b>

## Optimizing the dataset

renaming the columns for better implementation

In [92]:
df_ = df[['age', 'Diabetes', 'Taking Blood Pressure Medication', 'Smoke Tobacco', 'Systolic Blood Pressure', 'Gender', 'BMI']]
df_ = df_.rename(columns={'Taking Blood Pressure Medication': 'SBP_status', 'Diabetes': 'diabetes', 'Smoke Tobacco': 'smoking', 'Gender': 'gender', 'Systolic Blood Pressure':'SBP'})
df_.head()

,age,diabetes,SBP_status,smoking,SBP,gender,BMI
0,54,0,0,0,124.333333,female,25.14
1,62,1,0,0,124.666667,female,27.64
2,68,1,0,0,117.333333,male,20.13
3,47,1,0,0,132.000000,female,29.76
4,59,0,1,0,129.000000,male,30.07


### Normalizing the data for every patient

Yes or No will be 1 or 0 to make the formula works.

In [70]:
for column in df_.columns[1:4]:
    df_[column] = df_[column].apply(lambda x: 0 if x=='No' else 1)

<ipython-input-70-aaf93febda8f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_[column] = df_[column].apply(lambda x: 0 if x=='No' else 1)


Splitting the data into males and females as Framingham algorithms works differently with each gender.

In [105]:
df_male = df_[df_.gender == 'male']
df_female =df_[df_.gender == 'female']

Age should be normalised to get better results in the mentioned algorithm.

<b>Framinham algorithm only works in range 30 and 74 years old people. <br>
If the age reported is less than 30, we will consider it as 30. <br>
If the age reported is greater than 74, we will consider it as 74.<b>

In [104]:
df_.age = df_.age.apply(lambda x: 30 if x<30 else x)
df_.age = df_.age.apply(lambda x: 74 if x>74 else x)

### Women Health Risk Formula

This is the Women Health Risk Formula for the Framingham algorithm

$$0.94833^{\exp(\sum\beta X - 26.0145)}$$

In [ ]:
def formula_woman(sum_beta):
    return 1-np.power(0.94833, np.exp(sum_beta - 26.0145))

![women_hra.jpg](women_hra.jpg)

In [119]:
def sum_beta_woman(age, BMI, SBP, SBP_status, smoking, diabetes): 
    return np.log(age)*2.72107 + np.log(BMI)*0.51125 \
    + np.log(SBP)*2.81291*(1-SBP_status)  +  np.log(SBP)*2.88267*SBP_status \
    +  smoking*0.61868 + diabetes*0.77763

### Men Health Risk Formula
This is the men Health Risk Formula for the Framingham algorithm

$$0.88431^{\exp(\sum\beta X - 23.9388)}$$

In [47]:
def formula_man(sum_beta):
    return 1-np.power(0.88431, np.exp(sum_beta - 23.9388))

![men_hra.jpg](men_hra.jpg)

In [43]:
def sum_beta_man(age, BMI, SBP, SBP_status, smoking, diabetes): 
    return np.log(age)*3.11296 + np.log(BMI)*0.79277 \
    + np.log(SBP)*1.85508*(1-SBP_status)  +  np.log(SBP)*1.92672*SBP_status \
    +  smoking*0.70953 + diabetes*0.53160

## Calculating the HRA for every patient in the mDoc dataset

In [108]:
df_male['K_HRA%'] = df_male.apply(lambda x: formula_man(sum_beta_man(x.age, x.BMI, x.SBP, x.SBP_status, x.smoking, x.diabetes)), axis=1)*100

<ipython-input-108-2aef8bc08f65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male['K_HRA%'] = df_male.apply(lambda x: formula_man(sum_beta_man(x.age, x.BMI, x.SBP, x.SBP_status, x.smoking, x.diabetes)), axis=1)*100


In [109]:
df_female['K_HRA%'] = df_female.apply(lambda x: formula_woman(sum_beta_woman(x.age, x.BMI, x.SBP, x.SBP_status, x.smoking, x.diabetes)), axis=1)*100

<ipython-input-109-632455edee97>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female['K_HRA%'] = df_female.apply(lambda x: formula_woman(sum_beta_woman(x.age, x.BMI, x.SBP, x.SBP_status, x.smoking, x.diabetes)), axis=1)*100


In [110]:
df_male.head()

,age,diabetes,SBP_status,smoking,SBP,gender,BMI,K_HRA%
2,68,1,0,0,117.333333,male,20.13,27.175455
4,59,0,1,0,129.000000,male,30.07,24.275808
5,65,1,0,0,104.333333,male,25.62,23.533295
10,74,1,1,0,123.666667,male,24.84,53.180302
12,65,1,0,0,122.333333,male,28.70,32.592773


In [111]:
df_female.head()

,age,diabetes,SBP_status,smoking,SBP,gender,BMI,K_HRA%
0,54,0,0,0,124.333333,female,25.14,5.450401
1,62,1,0,0,124.666667,female,27.64,17.127171
3,47,1,0,0,132.000000,female,29.76,10.222170
6,48,1,0,0,126.000000,female,45.00,11.641502
7,52,0,0,0,123.666667,female,35.32,5.755136


### Conclusion

Now we have the algorithm available too work on mDoc data types. Most of mDoc patients do not report their cholesterol rates or theit triglycerides, so we were able to choose the best algorithm in order to make use of the available data. Framingham was our best algorithm to calculate accurate  predictions.

### The algorithm is live on the below link: <br>
<b>https://katakit-health-care.netlify.app/ <b><br><br>

The use of the algorithm, the long-term solution that we aim to implement, and the relevancy to mDoc mission and vision are listed on the above-mentioned website. <br>

## References

Brindle P, Emberson J, Lampe F, Walker M, Whincup P, Fahey T, Ebrahim S (Nov 2003). "Predictive accuracy of the Framingham coronary risk score in British men: prospective cohort study". BMJ. 327 (7426): 1267. doi:10.1136/bmj.327.7426.1267. PMC 286248. PMID 14644971.

Chun M, Clarke R, Cairns BJ, Clifton D, Bennett D, Chen Y, Guo Y, Pei P, Lv J, Yu C, Yang L, Li L, Chen Z, Zhu T; China Kadoorie Biobank Collaborative Group. Stroke risk prediction using machine learning: a prospective cohort study of 0.5 million Chinese adults. J Am Med Inform Assoc. 2021 Jul 30;28(8):1719-1727. doi: 10.1093/jamia/ocab068. PMID: 33969418; PMCID: PMC8324240.


Reddy G. T, Srivatsava A, Lakshmanna K, Kaluri R, Karnam S, Nagaraja G. Risk Prediction to Examine Health Status with Real and Synthetic Datasets. Biomed Pharmacol J 2017;10(4).

Ridker PM, Buring JE, Rifai N, Cook NR. Development and validation of improved algorithms for the assessment of global cardiovascular risk in women: The Reynolds Risk Score. JAMA 2007;297:611-619

Ridker PM, Buring JE, Rifai N, Cook NR. Development and validation of improved algorithms for the assessment of global cardiovascular risk in women: the Reynolds Risk Score. JAMA. 2007 Feb 14;297(6):611-9. doi: 10.1001/jama.297.6.611. Erratum in: JAMA. 2007 Apr 4;297(13):1433. PMID: 17299196.


Sacco RL, Khatri M, Rundek T, Xu Q, Gardener H, Boden-Albala B, Di Tullio MR, Homma S, Elkind MS, Paik MC (Dec 2009). "Improving global vascular risk prediction with behavioral and anthropometric factors. The multiethnic NOMAS (Northern Manhattan Cohort Study)". J Am Coll Cardiol. 54 (24): 2303–11. doi:10.1016/j.jacc.2009.07.047. PMC 2812026. PMID 19958966.

https://reference.medscape.com/calculator/37/acc-aha-cv-risk-calculator-2013

https://www.cvriskcalculator.com/

https://healthitanalytics.com/news/key-use-cases-for-health-risk-assessments

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6319275/

https://www.wellsource.com/health-risk-assessments

https://www.hsa.ie/eng/your_industry/construction/designing_for_safety/risk_assessment/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4310968/

https://par.nsf.gov/servlets/purl/10097838


https://doi.org/10.1161/01.cir.0000437741.48606.98

https://doi.org/10.1161/01.cir.0000437741.48606.98

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC286248/

https://jamanetwork.com/journals/jama/fullarticle/198843

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8108792/

https://www.framinghamheartstudy.org/fhs-risk-functions/cardiovascular-disease-10-year-risk/

https://jamanetwork.com/journals/jama/fullarticle/205528

https://library.ndsu.edu/ir/bitstream/handle/10365/28799/Health%20Risk%20Prediction%20Using%20Big%20Medical%20Data%20-%20a%20Collaborative%20Filtering-Enhanced%20Deep%20Learning%20Approach.pdf;jsessionid=D08D3B375BF4B92CE9D5BD029CF95E81?sequence=1

https://medlineplus.gov/lab-tests/heart-disease-risk-assessment/

http://www.reynoldsriskscore.org/

https://read.qxmd.com/read/24222018/2013-acc-aha-guideline-on-the-assessment-of-cardiovascular-risk-a-report-of-the-american-college-of-cardiology-american-heart-association-task-force-on-practice-guidelines

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0044347

http://www.reynoldsriskscore.org/

https://tools.acc.org/ascvd-risk-estimator-plus/#!/calculate/estimate/


_Made with ♥ by Katakit_ <br>
_Thank you,_